In [1]:
import pandas as pd
import numpy as np

In [2]:
from matplotlib import pyplot as plt
import seaborn as sns
sns.set()

In [3]:
from _plotly_future_ import v4_subplots
import plotly

In [4]:
plotly.offline.init_notebook_mode(connected=True)

In [5]:
%matplotlib inline

In [6]:
ratings = pd.read_csv("/home/jczestochowska/workspace/ada/chicago_food/data/processed/restaurants_with_ratings.csv")
inspections = pd.read_csv("/home/jczestochowska/workspace/ada/chicago_food/data/raw/food-inspections.csv")

In [7]:
inspections.head()

,Inspection ID,DBA Name,AKA Name,License #,Facility Type,Risk,Address,City,State,Zip,...,Results,Violations,Latitude,Longitude,Location,Historical Wards 2003-2015,Zip Codes,Community Areas,Census Tracts,Wards
0,2320315,SERENDIPITY CHILDCARE,SERENDIPITY CHILDCARE,2216009.0,Daycare Above and Under 2 Years,Risk 1 (High),1300 W 99TH ST,CHICAGO,IL,60643.0,...,Pass,NaN,41.714168,-87.655291,"{'longitude': '41.7141680989703', 'latitude': ...",NaN,NaN,NaN,NaN,NaN
1,2320342,YOLK TEST KITCHEN,YOLK TEST KITCHEN,2589655.0,Restaurant,Risk 1 (High),1767 N MILWAUKEE AVE,CHICAGO,IL,60647.0,...,Pass w/ Conditions,23. PROPER DATE MARKING AND DISPOSITION - Comm...,41.913588,-87.682203,"{'longitude': '41.9135877900482', 'latitude': ...",NaN,NaN,NaN,NaN,NaN
2,2320328,LAS ASADAS MEXICAN GRILL,LAS ASADAS MEXICAN GRILL,2583309.0,Restaurant,Risk 1 (High),3834 W 47TH ST,CHICAGO,IL,60632.0,...,Out of Business,NaN,41.808025,-87.720037,"{'longitude': '41.80802515275297', 'latitude':...",NaN,NaN,NaN,NaN,NaN
3,2320319,LA PALAPITA,LA PALAPITA,2694702.0,Restaurant,Risk 1 (High),3834 W 47TH ST,CHICAGO,IL,60632.0,...,Pass,47. FOOD & NON-FOOD CONTACT SURFACES CLEANABLE...,41.808025,-87.720037,"{'longitude': '41.80802515275297', 'latitude':...",NaN,NaN,NaN,NaN,NaN
4,2320228,47TH ST CANTINA,47TH ST CANTINA,2678250.0,Liquor,Risk 3 (Low),4311 W 47TH ST,CHICAGO,IL,60632.0,...,Pass w/ Conditions,"3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL E...",41.807662,-87.731480,"{'longitude': '41.80766199360051', 'latitude':...",NaN,NaN,NaN,NaN,NaN


In [8]:
inspections.columns

Index(['Inspection ID', 'DBA Name', 'AKA Name', 'License #', 'Facility Type',
       'Risk', 'Address', 'City', 'State', 'Zip', 'Inspection Date',
       'Inspection Type', 'Results', 'Violations', 'Latitude', 'Longitude',
       'Location', 'Historical Wards 2003-2015', 'Zip Codes',
       'Community Areas', 'Census Tracts', 'Wards'],
      dtype='object')

We won't need most of the columns in this part, let's drop them

In [9]:
inspections.drop(labels=["License #", "Facility Type", "Address", "City", "State","Wards", "AKA Name", "Location", "Historical Wards 2003-2015", "Inspection Type", "Latitude", "Longitude", "Zip Codes", "Zip", "Community Areas","Census Tracts", "Violations"], axis=1, inplace=True)

In [10]:
inspections["DBA Name"] = inspections["DBA Name"].apply(str.lower)

First we calculate inspection score for those restaurants for which we have ratings

In [11]:
merged = inspections.merge(ratings, how="inner", left_on="DBA Name", right_on="place_name")

In [12]:
merged.head()

,Inspection ID,DBA Name,Risk,Inspection Date,Results,Unnamed: 0,place_id,place_name,latitude,longitude,address,zip_code,rating,total_number_of_ratings
0,2320342,yolk test kitchen,Risk 1 (High),2019-10-23T00:00:00.000,Pass w/ Conditions,0,ChIJleWegJHTD4gRsIMGm3IjUcM,yolk test kitchen,41.913588,-87.682203,1767 n milwaukee ave,60647.0,4.4,180.0
1,2166304,yolk test kitchen,Risk 1 (High),2018-04-18T00:00:00.000,Pass,0,ChIJleWegJHTD4gRsIMGm3IjUcM,yolk test kitchen,41.913588,-87.682203,1767 n milwaukee ave,60647.0,4.4,180.0
2,2159782,yolk test kitchen,Risk 1 (High),2018-04-10T00:00:00.000,Not Ready,0,ChIJleWegJHTD4gRsIMGm3IjUcM,yolk test kitchen,41.913588,-87.682203,1767 n milwaukee ave,60647.0,4.4,180.0
3,2320328,las asadas mexican grill,Risk 1 (High),2019-10-23T00:00:00.000,Out of Business,1,ChIJI1dehI0xDogRRkvKIpUspHM,las asadas mexican grill,41.808025,-87.720037,3834 w 47th st,60632.0,4.0,166.0
4,2303290,las asadas mexican grill,Risk 1 (High),2019-07-17T00:00:00.000,Pass w/ Conditions,1,ChIJI1dehI0xDogRRkvKIpUspHM,las asadas mexican grill,41.808025,-87.720037,3834 w 47th st,60632.0,4.0,166.0


In [13]:
merged.drop(labels=["Unnamed: 0", "place_id","DBA Name"], axis=1, inplace=True)

In [14]:
merged.head()

,Inspection ID,Risk,Inspection Date,Results,place_name,latitude,longitude,address,zip_code,rating,total_number_of_ratings
0,2320342,Risk 1 (High),2019-10-23T00:00:00.000,Pass w/ Conditions,yolk test kitchen,41.913588,-87.682203,1767 n milwaukee ave,60647.0,4.4,180.0
1,2166304,Risk 1 (High),2018-04-18T00:00:00.000,Pass,yolk test kitchen,41.913588,-87.682203,1767 n milwaukee ave,60647.0,4.4,180.0
2,2159782,Risk 1 (High),2018-04-10T00:00:00.000,Not Ready,yolk test kitchen,41.913588,-87.682203,1767 n milwaukee ave,60647.0,4.4,180.0
3,2320328,Risk 1 (High),2019-10-23T00:00:00.000,Out of Business,las asadas mexican grill,41.808025,-87.720037,3834 w 47th st,60632.0,4.0,166.0
4,2303290,Risk 1 (High),2019-07-17T00:00:00.000,Pass w/ Conditions,las asadas mexican grill,41.808025,-87.720037,3834 w 47th st,60632.0,4.0,166.0


Get inspection scores

In [15]:
merged.Results.unique()

array(['Pass w/ Conditions', 'Pass', 'Not Ready', 'Out of Business',
       'No Entry', 'Fail', 'Business Not Located'], dtype=object)

We will only consider inspections with Pass w/ conditions, Pass and Fail status

In [16]:
merged = merged[merged.Results.isin(['Pass w/ Conditions', 'Pass', "Fail"])]

In [17]:
def get_score(result):
    if result == 'Pass w/ Conditions':
        score = 0.5
    elif result == 'Pass':
        score = 1
    elif result == 'Fail':
        score = 0
    return score

In [18]:
merged["inspection_score"] = merged.Results.apply(lambda x: get_score(x))

In [19]:
inspection_scores = merged.groupby(by="place_name")["inspection_score"].mean()

In [20]:
inspection_scores = inspection_scores.to_frame().reset_index()

In [21]:
len(inspection_scores)

5281

In [22]:
len(ratings)

5700

In [23]:
inspection_scores.head()

,place_name,inspection_score
0,'luna empanada shop',0.50
1,(k) new restaurant,1.00
2,1 n wacker kitchen and bar,0.50
3,"1,200 sq.ft. - yemen restaurant, inc.",0.75
4,"1,4,3,7 soul restaurant",0.50


In [24]:
scores_ratings = inspection_scores.merge(ratings, how="left", left_on="place_name", right_on="place_name")[["place_name", "inspection_score", "rating"]]

In [25]:
scores_ratings.head()

,place_name,inspection_score,rating
0,'luna empanada shop',0.50,4.7
1,(k) new restaurant,1.00,4.7
2,1 n wacker kitchen and bar,0.50,4.1
3,"1,200 sq.ft. - yemen restaurant, inc.",0.75,4.2
4,"1,4,3,7 soul restaurant",0.50,4.4


In [26]:
len(scores_ratings)

5291

In [27]:
pearson = scores_ratings.rating.corr(scores_ratings.inspection_score)
print(f"Inspection scores and rating have pearson correlation coefficient equal to {pearson}")

Inspection scores and rating have pearson correlation coefficient equal to -0.0035614202784061966


In [28]:
spearman = scores_ratings.rating.corr(scores_ratings.inspection_score, method='spearman')
print(f"Inspection scores and ratings have spearman correlation coefficient equal to {spearman}")

Inspection scores and ratings have spearman correlation coefficient equal to 0.02411536583968173


In [29]:
from plotly.subplots import make_subplots
import plotly.graph_objs as go

fig = make_subplots(rows=1, cols=2)

fig.add_trace(
    go.Violin(y=scores_ratings.rating, box_visible=True, line_color='black',
                   meanline_visible=True, fillcolor='orange', opacity=0.8,
                   name="Rating"),
    row=1, col=1
)

fig.add_trace(
    go.Violin(y=scores_ratings.inspection_score, box_visible=True, line_color='black',
                   meanline_visible=True, fillcolor='#2828a4', opacity=0.8,
                   name="Inspection Score"),
    row=1, col=2
)

fig.layout.update(height=600, width=800, 
                  title_text="Violin plots of ratings and inspection scores",
                  plot_bgcolor='rgb(255,255,255)')
fig.show()

In [30]:
import plotly.offline as py
import plotly.figure_factory as ff

colorscale = ['#000000', '#3333cc', 'rgb(111, 111, 220)', '#adadeb', (0.98,0.98,0.98)]

fig = ff.create_2d_density(
    scores_ratings.inspection_score,
    scores_ratings.rating, 
    colorscale=colorscale,
    hist_color='#2828a4', 
    point_size=3
)


fig.layout.update(plot_bgcolor='rgb(255,255,255)',
                  title="Ratings and Inspection Score joint distribution",
                  xaxis_title="Inspection Score",
                  yaxis_title="Rating")

py.iplot(fig, filename='histogram_subplots')